In [2]:
import itertools
import os
import shutil
import contextlib
import pathlib
import json
import time

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import run
import environment
import evolve
import encode
import plot
from utils import ClassEncoder

In [3]:
def generate_argument_variations(arguments):
    keys = arguments.keys()
    values = arguments.values()
    variations = list(itertools.product(*values))
    argument_variations = []

    for variation in variations:
        argument_variation = dict(zip(keys, variation))

        results_dir = generate_results_dir(argument_variation)
        argument_variation['results_dir'] = results_dir

        argument_variations.append(argument_variation)

    return argument_variations


def generate_results_dir(arguments):
    selected_keys = ['iterations', 'genome_size', 'elitism', 'remove_dead_organisms', 'generation_time', 'food_energy', 'mutation', 'crossover']
    dir_name = "result_"
    for key, value in arguments.items():
        if key not in selected_keys:
            continue
        if isinstance(value, str):
            dir_name += f"{key}_{value.replace(',', '-')}_"
        elif isinstance(value, (int, float)):
            dir_name += f"{key}_{str(value).replace(',', '-')}_"
        elif isinstance(value, list):
            dir_name += f"{key}_{'-'.join(str(v) for v in value)}_"
        elif isinstance(value, (evolve.Selection, evolve.Crossover, evolve.Mutation)):
            dir_name += f"{value.__class__.__name__}_"
        else:
            dir_name += f"{key}_"
    
    return os.path.join('results', dir_name[:-1])  # Remove the trailing underscore

In [8]:
organism_vision_range = 1

vision = environment.SectorVision(
    distance=organism_vision_range,
    distance_sectors=4,
    angle_sectors=4,
)
genome_shapes = [
  [vision.organism_input_shape, 6, 2],
  # [vision.organism_input_shape, 12, 2],
  # [vision.organism_input_shape, 12, 6, 2],
  # [vision.organism_input_shape, 24, 12, 2],
]

fitness = evolve.EnergyFitness()

generation_time = 80
generations = 100

arguments = {
  'start_organism_number': [40],
  'width': [20],
  'height': [20],
  'iterations': [generation_time * generations],
  'generation_time': [generation_time],
  'organism_size': [0.12],
  'food_size': [0.05],
  'organism_vision_range': [organism_vision_range],
  'vision': [vision],
  'food_energy': [2],
  'food_appearance_number_rate': [0.7],
  'energy_decrease_rate': [0.008],
  'encoding': [encode.RealValued()],
  'selection': [
    # evolve.TruncationSelection(fitness=fitness, n=10),
    # evolve.TruncationSelection(fitness=fitness, n=15),
    evolve.TruncationSelection(fitness=fitness, n=20),
  ],
  'crossover': [
    evolve.SBXCrossover(n=2),
    # evolve.SBXCrossover(n=8),
    evolve.ArithmeticCrossover(),
    evolve.BLXCrossover(alpha=0.5),
    # evolve.SBXCrossover(n=12),
  ],
  'mutation': [
    evolve.GaussianMutation(mu=0, sigma=0.1, p=0.1),
    evolve.NonUniformMutation(b=5, p=0.05, T=80*100), # predefined number of iterations 80 * 60
    evolve.UniformMutation(low=-0.5, high=0.5, p=0.05),
  ],
  'elitism': [20],
  'genome_size': genome_shapes,
  'food_particles_at_start': [40],
  'remove_dead_organisms': [False],
}


variations = generate_argument_variations(arguments)

print(f'Total number of configurations (variations): {len(variations)}')

Total number of configurations (variations): 1


In [9]:
def run_organisms_environment(variation):
  if os.path.exists(variation['results_dir']):
    variation['results_dir'] += str(time.time())[-3:]

  with contextlib.suppress(Exception):
      os.mkdir(variation['results_dir'])
      
  metadata_filename = os.path.join(variation['results_dir'], 'metadata.json')
  with open(metadata_filename, 'w') as file:
    json.dump(variation, file, cls=ClassEncoder)

  run_tool = run.OrganismsSimpleEnvironmentRunTool(**variation)
  run_tool.run()

In [10]:
for i, var in enumerate(variations):
  print(f'Running sample №{i+1}/{len(variations)}')
  run_organisms_environment(var)

Running sample №1/1


Simulating organisms: 100%|██████████| 2400/2400 [11:07<00:00,  3.59it/s, Number of organisms=40, Gen=29]


In [11]:
# OPTIONAL CREATE ANIMATION
# picked_results_dir = 'results/result_iterations_2400_generation_time_80_food_energy_2_ArithmeticCrossover_NonUniformMutation_elitism_20_genome_size_16-6-2_remove_dead_organisms_False/'

# org_loc = pq.read_table(os.path.join(picked_results_dir,
#                                      'organisms_locations')).to_pandas()
# food_loc = pq.read_table(os.path.join(picked_results_dir,
#                                       'food_locations')).to_pandas()

# org_loc.reset_index(drop=True, inplace=True)
# food_loc.reset_index(drop=True, inplace=True)

# frames_dir = os.path.join(picked_results_dir, 'frames')
# frames_dir = pathlib.Path(picked_results_dir, 'frames')
# frames_dir.mkdir()
# plot.create_frames(org_loc, food_loc, arguments['width'][0], arguments['height'][0], arguments['organism_size'][0], arguments['food_size'][0], frames_dir)
# plot.generate_video(
#     os.path.join(picked_results_dir, 'frames/'),
#     framerate=24,
#     output=f'{picked_results_dir}evolution.mp4',
)

 99%|█████████▉| 2380/2399 [13:12<00:06,  3.00it/s]
ffmpeg version n6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.1.1 (GCC) 20230429
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enab

## Generation with steady-state genetic algorithm

In [ ]:
organism_vision_range = 1

vision = environment.SectorVision(
    distance=organism_vision_range,
    distance_sectors=4,
    angle_sectors=4,
)
genome_shapes = [
  [vision.organism_input_shape, 6, 2],
  # [vision.organism_input_shape, 12, 2],
  [vision.organism_input_shape, 12, 6, 2],
  # [vision.organism_input_shape, 24, 12, 2],
]

fitness = evolve.EnergyFitness()

generation_time = 2 # as we are using steady-state GA in this implementation we need to set a small number of generation_time
iterations = 8000

arguments_steady = {
  'start_organism_number': [40],
  'width': [20],
  'height': [20],
  'iterations': [iterations],
  'generation_time': [generation_time],
  'organism_size': [0.12],
  'food_size': [0.05],
  'organism_vision_range': [organism_vision_range],
  'vision': [vision],
  'food_energy': [2],
  'food_appearance_number_rate': [0.7],
  'energy_decrease_rate': 0.008,
  'encoding': [encode.RealValued()],
  'selection': [
    evolve.TruncationSelection(fitness=fitness, n=20),
  ],
  'crossover': [
    evolve.SBXCrossover(n=8),
    evolve.ArithmeticCrossover(),
    evolve.BLXCrossover(alpha=0.5),
  ],
  'mutation': [
    evolve.GaussianMutation(mu=0, sigma=0.1, p=0.1),
    evolve.NonUniformMutation(b=5, p=0.05, T=80*100), # predefined number of iterations 80 * 60
    evolve.UniformMutation(low=-0.5, high=0.5, p=0.05),
  ],
  'elitism': [20],
  'genome_size': genome_shapes,
  'food_particles_at_start': [40],
  'remove_dead_organisms': [True],
}


variations_steady = generate_argument_variations(arguments_steady)

print(f'Total number of configurations (variations): {len(variations_steady)}')

In [ ]:
def run_organisms_environment(variation):
  if os.path.exists(variation['results_dir']):
    variation['results_dir'] += str(time.time())[-3:]

  with contextlib.suppress(Exception):
      os.mkdir(variation['results_dir'])
      
  metadata_filename = os.path.join(variation['results_dir'], 'metadata.json')
  with open(metadata_filename, 'w') as file:
    json.dump(variation, file, cls=ClassEncoder)

  run_tool = run.OrganismsSimpleEnvironmentRunTool(**variation)
  run_tool.run()

In [ ]:
for i, var in enumerate(variations_steady):
  print(f'Running sample №{i+1}/{len(variations_steady)}')
  run_organisms_environment(var)

Running sample №1/4


Simulating organisms: 100%|██████████| 5001/5001 [14:51<00:00,  5.61it/s, Number of organisms=31, Gen=1250]


Running sample №2/4


Simulating organisms: 100%|██████████| 5001/5001 [15:14<00:00,  5.47it/s, Number of organisms=31, Gen=1250]


Running sample №3/4


Simulating organisms: 100%|██████████| 5001/5001 [15:07<00:00,  5.51it/s, Number of organisms=31, Gen=1250]


Running sample №4/4


Simulating organisms: 100%|██████████| 5001/5001 [15:36<00:00,  5.34it/s, Number of organisms=31, Gen=1250]
